# Kaggle
## Competition NFL Big Data Bowl

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Carregando os pacotes
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Statistic lib
from scipy import stats
from scipy.stats import skew, norm

# Sklearn lib
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

# Models
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from mlxtend.regressor import StackingCVRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
import xgboost as XGB
from sklearn.cluster import KMeans
import tqdm

# Misc lib
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from functools import partial
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from IPython.display import Image

# Utils
import pandasql as ps
import re 
import math, string, os
import datetime

# Options
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000
pd.set_option('display.max_columns', None)
import gc
gc.enable()

In [ ]:
# Carregando os dados de treino
train = pd.read_csv('../data/train.csv')
#train = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)
print ("Data is ready !!")

# EXPLICAÇÃO DO NFL

In [ ]:
from IPython.display import Image
Image(url = '../images/NFL_Image_01.png')

# 1. ANALISE GERAL

In [ ]:
# Visualizando os primeiros registros do dataset
train.head()

In [ ]:
# Visualizando os tipos das features
train.dtypes

In [ ]:
# Visualizando dados estatisticos das variaveis numericas
train.describe().T

In [ ]:
eda_df = train.copy()

# 2. DATA MISSING

In [ ]:
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100,2)})
    
    return dict_x

In [ ]:
# Verificando as colunas com dados missing do dataset de treino
missing = percent_missing(eda_df)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:50]

In [ ]:
# Setup do plot
sns.set_style("white")
f, ax = plt.subplots(figsize=(8, 7))
sns.set_color_codes(palette='deep')

# Identificando os valores missing
missing = round(eda_df.isnull().mean()*100,2)
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar(color="b")

# Visual presentation
ax.xaxis.grid(False)
ax.set(ylabel="Percent of missing values")
ax.set(xlabel="Features")
ax.set(title="Percent missing data by feature")
sns.despine(trim=True, left=True)

# 3. Criando as funções auxiliares de limpeza e conversao

In [ ]:
# Funcao para tratar os dados missing de cada variavel
def fill_na(data):
    data['WindDirection'].fillna('unknown',inplace=True)
    data['OffenseFormation'].fillna('unknown',inplace=True)
    data['StadiumType'].fillna('unknown',inplace=True)
    data['GameWeather'].fillna('unknown',inplace=True)
    data['FieldPosition'].fillna('NA',inplace=True)
    
    data['Temperature'].fillna(data['Temperature'].mean(), inplace=True)
    data['Humidity'].fillna(data['Humidity'].mean(), inplace=True)
    data['DefendersInTheBox'].fillna(math.ceil(data['DefendersInTheBox'].mean()),inplace=True)
    
# Funcao para agrupar as descricoes dos tipos de estadio
def agrupar_tipo_estadio(StadiumType):
    outdoor       = ['Outdoor', 'Outdoors', 'Cloudy', 'Heinz Field', 'Outdor', 'Ourdoor', 'Outside', 'Outddors', 'Outdoor Retr Roof-Open', 'Oudoor', 'Bowl']
    indoor_closed = ['Indoors', 'Indoor', 'Indoor, Roof Closed', 'Indoor, Roof Closed', 'Retractable Roof', 'Retr. Roof-Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed']
    indoor_open   = ['Indoor, Open Roof', 'Open', 'Retr. Roof-Open', 'Retr. Roof - Open']
    dome_closed   = ['Dome', 'Domed, closed', 'Closed Dome', 'Domed', 'Dome, closed']
    dome_open     = ['Domed, Open', 'Domed, open']
    
    if StadiumType in outdoor:
        return 'outdoor'
    elif StadiumType in indoor_closed:
        return 'indoor_closed'
    elif StadiumType in indoor_open:
        return 'indoor_open'
    elif StadiumType in dome_closed:
        return 'dome_closed'
    elif StadiumType in dome_open:
        return 'dome_open'
    else:
        return 'unknown' # se for n/a
    
# Funcao para agrupar as descricoes dos estadios
def agrupar_estadio(Stadium):

    if Stadium == 'Broncos Stadium at Mile High':
        return 'Broncos Stadium At Mile High'
    
    elif Stadium in ('CenturyField', 'CenturyLink'):
        return 'CenturyLink Field'
    
    elif Stadium == 'EverBank Field':
        return 'Everbank Field'
    
    elif Stadium in ('FirstEnergy', 'FirstEnergy Stadium', 'FirstEnergyStadium'):
        return 'First Energy Stadium'
   
    elif Stadium == 'Lambeau field':
        return 'Lambeau Field'

    elif Stadium == 'Los Angeles Memorial Coliesum':
        return 'Los Angeles Memorial Coliseum'
    
    elif Stadium in ('M & T Bank Stadium', 'M&T Stadium'):
        return 'M&T Bank Stadium'

    elif Stadium in ('Mercedes-Benz Dome', 'Mercedes-Benz Superdome'):
        return 'Mercedes-Benz SuperDome'
    
    elif Stadium in ('MetLife Stadium', 'Metlife Stadium', 'MetLife'):
        return 'MetLife Stadium' 
    
    elif Stadium == 'NRG':
        return 'NRG Stadium' 

    elif Stadium == 'Oakland-Alameda County Coliseum':
        return 'Oakland Alameda-County Coliseum' 
    
    elif Stadium == 'Paul Brown Stdium':
        return 'Paul Brown Stadium' 

    elif Stadium == 'Twickenham':
        return 'Twickenham Stadium' 
    
    else:
        return Stadium
    
# Funcao para agrupar a localizacao do estadio e do jogo
def agrupar_local(Location):

    if Location == "Arlington, Texas":
        return "Arlington, TX"
    elif Location in ("Baltimore, Maryland","Baltimore, Md."):
        return "Baltimore, MD"
    elif Location == "Charlotte, North Carolina":
        return "Charlotte, NC"
    elif Location == "Chicago. IL":
        return "Chicago, IL"
    elif Location == "Cincinnati, Ohio":
        return "Cincinnati, OH"
    elif Location in ("Cleveland","Cleveland Ohio","Cleveland, Ohio","Cleveland,Ohio"):
        return "Cleveland, OH"
    elif Location == "Detroit":
        return "Detroit, MI"
    elif Location == "E. Rutherford, NJ" or Location == "East Rutherford, N.J.":
        return "East Rutherford, NJ"
    elif Location == "Foxborough, Ma":
        return "Foxborough, MA"
    elif Location == "Houston, Texas":
        return "Houston, TX"
    elif Location in ("Jacksonville Florida","Jacksonville, Fl","Jacksonville, Florida"):
        return "Jacksonville, FL"
    elif Location == "London":
        return "London, England"
    elif Location == "Los Angeles, Calif.":
        return "Los Angeles, CA"
    elif Location == "Miami Gardens, Fla.":
        return "Miami Gardens, FLA"
    elif Location in ("New Orleans","New Orleans, La."):
        return "New Orleans, LA"
    elif Location == "Orchard Park NY":
        return "Orchard Park, NY"
    elif Location == "Philadelphia, Pa.":
        return "Philadelphia, PA"
    elif Location == "Pittsburgh":
        return "Pittsburgh, PA"
    elif Location == "Seattle":
        return "Seattle, WA"
    else:
        return Location
    
# Funcao para agrupar o gramado do estadio
def agrupar_gramado(Turf):
    if Turf == 'Artifical':
        return 'Artificial'
    
    elif Turf in ('FieldTurf', 'Field turf'):
        return 'Field Turf'

    elif Turf in ('FieldTurf360', 'FieldTurf 360'):
        return 'Field Turf 360'

    elif Turf in ('Natural', 'Natural grass', 'Naturall Grass', 'grass', 'natural grass', 'SISGrass', 'Natural Grass'):
        return "Grass"

    elif Turf == "UBU Sports Speed S5-M":
        return "UBU Speed Series-S5-M"

    else:
        return Turf

# Funcao para agrupar os dados de direcao do vento
def agrupa_wind_direction(WindDirection):
    wd = str(WindDirection).upper()
    
    if wd == 'N' or 'FROM N' in wd:
        return 'north'
    if wd == 'S' or 'FROM S' in wd:
        return 'south'
    if wd == 'W' or 'FROM W' in wd:
        return 'west'
    if wd == 'E' or 'FROM E' in wd:
        return 'east'
    
    if 'FROM SW' in wd or 'FROM SSW' in wd or 'FROM WSW' in wd:
        return 'south west'
    if 'FROM SE' in wd or 'FROM SSE' in wd or 'FROM ESE' in wd:
        return 'south east'
    if 'FROM NW' in wd or 'FROM NNW' in wd or 'FROM WNW' in wd:
        return 'north west'
    if 'FROM NE' in wd or 'FROM NNE' in wd or 'FROM ENE' in wd:
        return 'north east'
    
    if 'NW' in wd or 'NORTHWEST' in wd:
        return 'north west'
    if 'NE' in wd or 'NORTH EAST' in wd:
        return 'north east'
    if 'SW' in wd or 'SOUTHWEST' in wd:
        return 'south west'
    if 'SE' in wd or 'SOUTHEAST' in wd:
        return 'south east'

    return 'unknown'

# Funcao para agrupar as descricoes de clima
def agrupar_clima(GameWeather):
    chuva   = ['Rainy', 'Rain Chance 40%', 'Showers',
               'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
               'Scattered Showers', 'Cloudy, Rain', 'Rain shower', 'Light Rain', 'Rain']
    nublado = ['Cloudy, light snow accumulating 1-3"', 'Party Cloudy', 'Cloudy, chance of rain',
               'Coudy', 'Cloudy, 50% change of rain', 'Rain likely, temps in low 40s.',
               'Cloudy and cold', 'Cloudy, fog started developing in 2nd quarter',
               'Partly Clouidy', '30% Chance of Rain', 'Mostly Coudy', 'Cloudy and Cool',
               'cloudy', 'Partly cloudy', 'Overcast', 'Hazy', 'Mostly cloudy', 'Mostly Cloudy',
               'Partly Cloudy', 'Cloudy']
    limpo   = ['Partly clear', 'Sunny and clear', 'Sun & clouds', 'Clear and Sunny',
               'Sunny and cold', 'Sunny Skies', 'Clear and Cool', 'Clear and sunny',
               'Sunny, highs to upper 80s', 'Mostly Sunny Skies', 'Cold',
               'Clear and warm', 'Sunny and warm', 'Clear and cold', 'Mostly sunny',
               'T: 51; H: 55; W: NW 10 mph', 'Clear Skies', 'Clear skies', 'Partly sunny',
               'Fair', 'Partly Sunny', 'Mostly Sunny', 'Clear', 'Sunny']
    neve    = ['Heavy lake effect snow', 'Snow']
    none    = ['N/A Indoor', 'Indoors', 'Indoor', 'N/A (Indoors)', 'Controlled Climate']

    
    if GameWeather in chuva:
        return 'chuva'
    elif GameWeather in nublado:
        return 'nublado'
    elif GameWeather in limpo:
        return 'limpo'
    elif GameWeather in neve:
        return 'neve'
    elif GameWeather in none:
        return 'none'
    else:
        return 'none' # se for n/a
    
# Funcao para converter a velocidade do vento
def convert_wind_speed(WindSpeed):
    ws = str(WindSpeed)

    if ws.isdigit():
        return int(ws)

    if '-' in ws:
        return int(ws.split('-')[0])

    if ws.split(' ')[0].isdigit():
        return int(ws.split(' ')[0])

    if 'mph' in ws.lower():
        return int(ws.lower().split('mph')[0])
    else:
        return 0
    
    
# Funcao para converter altura de feet-inches para centimetros
def convert_to_cm(ft_in):
    h_ft   = int(ft_in.split('-')[0])
    h_inch = int(ft_in.split('-')[1])
    h_inch += h_ft * 12
    h_cm = round(h_inch * 2.54, 1)
    #print("Your height is : %d cm." % h_cm)   
    
    return h_cm

# Funcao para converter peso em lbs para kg
def convert_to_kg(lbs):
    kg = lbs * 0.45359237
    #print("The weight is", kg, "in kilograms")
    
    return kg

# Funcao para converter temperatura Fahrenheit para Celsius
def convert_to_celsius(fah):
    celsius = (fah - 32) * 5.0/9.0
    #print("Temperature:", fah, "Fahrenheit = ", celsius, " C")
    return celsius

# Funcao para converter as features de data e extrair dia, mes, ano, hora, minuto, segundo
def convert_data(data):
    #data['PlayerBirthDate'] = pd.to_datetime(data['PlayerBirthDate'], "%m/%d/%Y")
    data['PlayerBirthDate'] = data['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    data['PlayerBirthDate_day'] = data['PlayerBirthDate'].dt.day.astype(int)
    data['PlayerBirthDate_month'] = data['PlayerBirthDate'].dt.month.astype(int)
    data['PlayerBirthDate_year'] = data['PlayerBirthDate'].dt.year.astype(int)

    data['TimeSnap'] = data['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    #data['TimeSnap'] = pd.to_datetime(data['TimeSnap'], "%Y-%m-%dT%H:%M:%S.%fZ")
    data['TimeSnap_min'] = data['TimeSnap'].dt.minute.astype(int)
    data['TimeSnap_seg'] = data['TimeSnap'].dt.second.astype(int)
    
    data['TimeHandoff'] = data['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    #data['TimeHandoff'] = pd.to_datetime(data['TimeHandoff'], "%Y-%m-%dT%H:%M:%S.%fZ")
    data['TimeHandoff_min'] = data['TimeHandoff'].dt.minute.astype(int)
    data['TimeHandoff_seg'] = data['TimeHandoff'].dt.second.astype(int)
    
    
    
# Funcao para converter uma string horario em segundos
def str_to_seconds(time):
    time = time.split(':')
    sec = int(time[0])*60 + int(time[1]) + int(time[2])/60
    return sec
    
# Funcao para criar um plot de distribuicao para cada feature
def plot_distribution(dataset, cols=5, width=20, height=25, hspace=0.4, wspace=0.5):
    """
    Plot distributions for each column in a dataset.
    Seaborn countplots are used for categorical data and distplots for numerical data

    args:
    ----
    dataset {dataframe} - the data that will be plotted
    cols {int} - how many distributions to plot for each row
    width {int} - how wide each plot should be
    height {int} - how tall each plot should be
    hspace {float} - horizontal space between plots
    wspace {float} - vertical space between plots 
    """
    # plot styling
    plt.style.use('fivethirtyeight')
    fig = plt.figure(figsize=(width, height))
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=wspace, hspace=hspace)
    # calculate rows needed
    rows = math.ceil(float(dataset.shape[1]) / cols)
    # create a countplot for top 20 categorical values
    # and a distplot for all numerical values
    for i, column in enumerate(dataset.columns):
        ax = fig.add_subplot(rows, cols, i + 1)
        ax.set_title(column)
        if dataset.dtypes[column] == np.object:
            # grab the top 10 for each countplot
            g = sns.countplot(y=column, 
                              data=dataset,
                              order=dataset[column].value_counts().index[:10])
            # make labels only 20 characters long and rotate x labels for nicer displays
            substrings = [s.get_text()[:20] for s in g.get_yticklabels()]
            g.set(yticklabels=substrings)
            plt.xticks(rotation=25)
        else:
            g = sns.distplot(dataset[column])
            plt.xticks(rotation=25)

# 4. Resolvendo missing values e realizando limpeza das features

In [ ]:
# Primeiro, vou preencher os dados missing e conversao das datas
fill_na(eda_df)
convert_data(eda_df)

# Executar a funcao para converter a altura do jogador em cm
eda_df['PlayerHeight'] = eda_df['PlayerHeight'].apply(convert_to_cm)

# Executar a funcao para converter a altura do jogador em cm
eda_df['PlayerWeight'] = eda_df['PlayerWeight'].apply(convert_to_kg)

# Executar a funcao para converter a temperatura em Celsius
eda_df['Temperature'] = eda_df['Temperature'].apply(convert_to_celsius)

# Executar a funcao para agrupar o nome dos tipos de estadios
eda_df['StadiumType'] = eda_df['StadiumType'].apply(agrupar_tipo_estadio)

# Executar a funcao de agrupar os nomes dos estadios
eda_df['Stadium'] = eda_df['Stadium'].apply(agrupar_estadio)

# Executar a funcao de agrupar a descricao dos locais dos jogos
eda_df['Location'] = eda_df['Location'].apply(agrupar_local)

# Executar a funcao de agrupar a descricao dos locais dos jogos
eda_df['Turf'] = eda_df['Turf'].apply(agrupar_gramado)

# Executar a funcao de agrupar a descricao da direção do vento
eda_df['WindDirection'] = eda_df['WindDirection'].apply(agrupa_wind_direction)

# Executar a funcao de converter a velocidade do vento para numerico
eda_df['WindSpeed'] = eda_df['WindSpeed'].apply(convert_wind_speed)

# Executar a funcao de agrupar a descricao do clima
eda_df['GameWeather'] = eda_df['GameWeather'].apply(agrupar_clima)

# Executar a funcao para converter o horario do jogo em segundos
eda_df['GameClock'] = eda_df['GameClock'].apply(str_to_seconds)

# 5. Analisando Correlacoes

In [ ]:
# Correlação de Pearson
cor_mat = eda_df.corr(method = 'pearson')

# Visualizando o grafico de heatmap
f, ax = plt.subplots(figsize=(18, 18))
sns.heatmap(cor_mat,linewidths=.1,fmt= '.3f',ax=ax,square=True,cbar=True,annot=True)

# 6. Analisando todas as features do dataset
- Neste momento as analises serao com base em um novo dataset
- Esse dataset será com base no jogador que está jogando
- Essa analise foi identificada depois de verificar as features NflId e NflIdRusher
- Essas duas features sao vinculadas, onde é possivel obter o jogador participante da jogada e obter a quantidade de jardas ganhas ou perdidas

In [ ]:
# a unique play identifier
# Sao 22 registros por jogada
# Pois sao 11 jogadores do time da casa e 11 jogadores do time visitante
eda_df['PlayId'].value_counts().head()

In [ ]:
# Vinculando o NFLid e NFLidRusher
train_jog = eda_df[eda_df['NflId'] == eda_df['NflIdRusher']]
print(train_jog.shape)

In [ ]:
# Quando realizado a ligacao entre NflId e NflIdRusher
# Verifica que agora cada jogada tem um unico registro
# e unica jarda obtida
train_jog['PlayId'].value_counts().head()

In [ ]:
# Exemplo mostrando a sequencia das jogadas de um jogo que contabilizou mudanca de jardas (+ ou -)
# É possivel identificar que essa sequencia é crescente, e registrada em PlayId
# Usando o filtro de apenas uma partida de 2017
train_jog[train_jog['GameId'] == 2017090700].head(10)

### Analisando a variavel target 'Yards'
- Extraindo os dados estatisticos, verifica a distribuição está entre -14 à 99
- Isso significa que em algumas jogadas existe um ganho de jardas e em outras perda de jardas
- Metade dos registros estao em 3 jardas ganhas

In [ ]:
# Descricao: o número de jardas ganha na jogada (variavel a ser prevista)
train_jog['Yards'].describe()

In [ ]:
# Analisando a variavel target 'Yards'
sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(18, 8))

# Fit a normal distribution
mu, std = norm.fit(train_jog["Yards"])

# Verificando a distribuicao de frequencia da variavel Yards
sns.distplot(train_jog["Yards"], color="b", fit = stats.norm)
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="Yards")
ax.set(title="Yards distribution: mu = %.2f,  std = %.2f" % (mu, std))
sns.despine(trim=True, left=True)

# Adicionando Skewness e Kurtosis
ax.text(x=1.1, y=1, transform=ax.transAxes, s="Skewness: %f" % train_jog["Yards"].skew(),\
        fontweight='demibold', fontsize=10, verticalalignment='top', horizontalalignment='right',\
        backgroundcolor='white', color='xkcd:poo brown')
ax.text(x=1.1, y=0.95, transform=ax.transAxes, s="Kurtosis: %f" % train_jog["Yards"].kurt(),\
        fontweight='demibold', fontsize=10, verticalalignment='top', horizontalalignment='right',\
        backgroundcolor='white', color='xkcd:dried blood')

plt.show()

### Analisando as variaveis do JOGADOR
- **PlayerHeight:** altura do jogador no formato feet-inches
- **PlayerWeight:** peso do jogador no formato lbs
- **PlayerBirthDate:** data de nascimento do jogador
- **PlayerCollegeName:** faculdade que o jogador frequentou
- **Position:** posição principal do jogador
- **JerseyNumber:** 
   - Refere se aos números do uniformes dos jogadores que vai de 1 a 99. Atacantes usam de  50 a 79 e nao tem permissao por exemplo para manipular uma jogada de scrimmage a não ser que reportem ao árbitro que estao jogando fora de posição

In [ ]:
# Visualizando o grafico de distribuicao para cada feature do jogador
columns_to_plot = ['PlayerHeight', 'PlayerWeight', 'PlayerBirthDate_year', 'PlayerCollegeName', 'JerseyNumber', 'Position']
plot_distribution(train_jog[columns_to_plot], cols=3, width=30, height=20, hspace=0.45, wspace=0.5)

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Altura do Jogador e Jardas
plt.figure(figsize=(18, 8))
sns.regplot(x='PlayerHeight', y='Yards', data=train_jog, color='b', x_jitter=1)
plt.xlabel('Altura do Jogador')
plt.ylabel('Jardas avancadas')
plt.title('Altura do Jogador vs Jardas', fontsize=20)
plt.show()

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Peso do Jogador e Jardas
plt.figure(figsize=(18, 8))
sns.regplot(x='PlayerWeight', y='Yards', data=train_jog, color='b', x_jitter=1)
plt.xlabel('Peso do Jogador')
plt.ylabel('Jardas avancadas')
plt.title('Peso do Jogador vs Jardas', fontsize=20)
plt.show()

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Altura e Peso do jogador
# É uma relacao de regressao positiva, onde quanto maais alto, mais pesado o jogador
plt.figure(figsize=(18, 8))
sns.regplot(x='PlayerHeight', y='PlayerWeight', data=train_jog, color='g', x_jitter=1)
plt.xlabel('Altura')
plt.ylabel('Peso')
plt.title('Altura vs Peso', fontsize=20)
plt.show()

In [ ]:
# Verificando jogadores acima de 140kg
# Encontrado apenas 1 jogador: Akiem Hicks
# A imagem abaixo mostra este jogador
train_jog[train_jog['PlayerWeight'] >= 140.00].head()

In [ ]:
from IPython.display import Image
Image(url = '../images/Akiem_Hicks.jpg',width=400,height=400)

In [ ]:
# Verificando jogadores com menos de 70kg
# Encontrado apenas 1 jogador: JoJo Natson
# A imagem abaixo mostra este jogador
train_jog[train_jog['PlayerWeight'] <= 70.00].head()

In [ ]:
#from IPython.display import Image
#Image(url = '../images/Jojo_Natson.jpg',width=400,height=400)

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Ano de nascimento do Jogador e Jardas
plt.figure(figsize=(18, 8))
sns.regplot(x='PlayerBirthDate_year', y='Yards', data=train_jog, color='b', x_jitter=2)
plt.xlabel('Ano')
plt.ylabel('Jardas avancadas')
plt.title('Ano de Nascimento do Jogador vs Jardas', fontsize=20)
plt.show()

In [ ]:
# Exibe o grafico de catPlot com a relacao entre o clima e Jardas
g = sns.catplot(x='PlayerCollegeName', y='Yards', hue='Season', data=train_jog, height=8, aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre JerseyNumber e Jardas
plt.figure(figsize=(18, 8))
sns.regplot(x='JerseyNumber', y='Yards', data=train_jog, color='b', x_jitter=2)
plt.xlabel('Jersey Number')
plt.ylabel('Jardas avancadas')
plt.title('Número Jersey vs Jardas', fontsize=20)
plt.show()

In [ ]:
train_jog['Position'].value_counts()

In [ ]:
# Exibe o grafico de catPlot com a relacao entre o Posição Principal do Jogador e Jardas
g = sns.catplot(x='Position', y='Yards', hue='Season', data=train_jog, kind="box", height=8, aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
# Exibe o grafico linear com a relacao entre Peso, Posição e Jardas
g = sns.lmplot(x='PlayerWeight', y='Yards', data=train_jog, 
               x_jitter=.2, col="Position" , height=6, aspect=1, col_wrap=5)

### Analisando as variaveis gerais da PARTIDA
- **Season:** ano da temporada
- **Week:** semana do jogo
- **Team:** detalhes dos times: donos da casa ou visitante (home or away)
- **StadiumType:** nome do tipo do estádio de futebol americano
- **Stadium:** nome do estádio de futebol americano
- **Location:**: localização do estádio
- **Turf:** superfície do gramado do estádio
- **GameClock:** hora da partida
- **HomeTeamAbbr:** abreviação do time da casa
- **VisitorTeamAbbr:** abreviação do time visitante

In [ ]:
# Visualizando o grafico de distribuicao para cada feature do jogador
columns_to_plot = ['Season', 'Week', 'Team', 'StadiumType', 'Stadium', 
                   'Location', 'Turf', 'GameClock', 'HomeTeamAbbr', 'VisitorTeamAbbr']
plot_distribution(train_jog[columns_to_plot], cols=3, width=30, height=20, hspace=0.45, wspace=0.5)

In [ ]:
# Exibe o grafico linear com a relacao entre Season e Jardas
g = sns.lmplot(x='Season', y='Yards', hue='Season', data=train_jog, x_jitter=.2, col="Season" , height=10)

In [ ]:
# Exibe o grafico linear com a relacao entre Season, Semana e Jardas
g = sns.lmplot(x='Week', y='Yards', hue='Season', data=train_jog, x_jitter=.2, col="Season" , height=10)

In [ ]:
# Season tem uma correlação de 1.0 com GameId e PlayId. Exemplo abaixo com 2017 e 2018:
# GameId e PlayId contem o ano e mais algumas informacoes
# Parece ser o mes e o dia e mais um identificador
# Talvez seria necessario deixar a variavel GameId e PlayId somente com o indicador individual
tmp = train_jog.set_index('Season')
print(tmp.loc[[2017], ['GameId','PlayId']].head(1))
print(tmp.loc[[2018], ['GameId','PlayId']].head(1))

In [ ]:
# MetLife Stadium só tem OUTDOOR
# Vou atribuir o tipo indefinido para outdoor
from IPython.display import Image
Image(url = '../images/MetLife_Stadium.jpeg',width=400,height=400)

In [ ]:
# StubHub Center so tem OUTDOOR
# Vou atribuir o tipo indefinido para outdoor
from IPython.display import Image
Image(url = '../images/StubHub_center.jpg',width=400,height=400)

In [ ]:
# Corrigindo a feature Stadium
train_jog.loc[train_jog['Stadium'] == 'MetLife Stadium', 'StadiumType'] = 'outdoor'
train_jog.loc[train_jog['Stadium'] == 'StubHub Center', 'StadiumType'] = 'outdoor'

In [ ]:
# Vista do estadio Mercedes_Benz onde tem varios tipos de modos de jogo
from IPython.display import Image
Image(url = '../images/Mercedes_Benz Stadium.jpg',width=400,height=400)

In [ ]:
# Exibe o grafico linear com a relacao entre Semana, Tipo de Estadio, Temporada e Jardas
g = sns.lmplot(x='Week', y='Yards', hue='Season', data=train_jog, 
               x_jitter=.2, col="StadiumType" , height=8, col_wrap=3)

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Estadio e Jardas
g = sns.catplot(x='Stadium', y='Yards', hue='Season', data=train_jog, 
                kind="violin", split=True, height=8, aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Estadio e Jardas
g = sns.catplot(x='Location', y='Yards', hue='Season', data=train_jog, 
                kind="violin", split=True, height=8, aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
# description of the field surface
# Tipos de gramados dos estadios
train_jog.groupby('Turf').size()

In [ ]:
# Exibe o grafico de catPlot com a relacao entre Estadio e Jardas
g = sns.catplot(x='Turf', y='Yards', hue='Season', data=train_jog, kind="box", height=8, aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
# Exibe o grafico de distribuicao da feature GameClock
g = sns.distplot(train_jog['GameClock'])

### Analisando as variaveis gerais do AMBIENTE
- **GameWeather:** clima
- **Temperature:** temperatura ambiente (em Fahrenheit)
- **Humidity:** umidade do ar
- **WindDirection:** direção do vento
- **WindSpeed:** velocidade do vento

In [ ]:
# Visualizando o grafico de distribuicao para cada feature do ambiente
columns_to_plot = ['GameWeather', 'Temperature', 'Humidity', 'WindDirection', 'WindSpeed']
plot_distribution(train_jog[columns_to_plot], cols=3, width=30, height=20, hspace=0.45, wspace=0.5)

In [ ]:
# Exibe o grafico de catPlot com a relacao entre o clima e Jardas
g = sns.catplot(x='GameWeather', y='Yards', hue='Season', data=train_jog, kind="box", height=8, aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Temperatura e Jardas
plt.figure(figsize=(18, 8))
sns.regplot(x='Temperature', y='Yards', data=train_jog, color='b', x_jitter=2)
plt.xlabel('Temperatura')
plt.ylabel('Jardas avancadas')
plt.title('Temperatura(Celsius) vs Jardas', fontsize=20)
plt.show()

In [ ]:
# Exibe o grafico linear com a relacao entre Temperatura, Clima e Jardas
g = sns.lmplot(x='Temperature', y='Yards', data=train_jog, 
               x_jitter=.2, col="GameWeather" , height=6, aspect=1)

In [ ]:
# Tem muitos registros com umidade 0
# Talvez seja melhor fazer um tratamento
train_jog['Humidity'].value_counts().head()

In [ ]:
train_jog['Humidity'].describe()

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Temperatura e Jardas
plt.figure(figsize=(18, 8))
sns.regplot(x='Humidity', y='Yards', data=train_jog, color='b', x_jitter=2)
plt.xlabel('Umidade')
plt.ylabel('Jardas avancadas')
plt.title('Umidade do Ar vs Jardas', fontsize=20)
plt.show()

In [ ]:
# Exibe o grafico linear com a relacao entre Umidade, Clima e Jardas
# A maioria da umidade 0 está no clima NONE
g = sns.lmplot(x='Humidity', y='Yards', data=train_jog, 
               x_jitter=.2, col="GameWeather" , height=6, aspect=1)

In [ ]:
train_jog['WindDirection'].value_counts()

In [ ]:
# Exibe o grafico linear com a relacao entre Temperatura, Direcao do Vento e Jardas
g = sns.lmplot(x='Temperature', y='Yards', data=train_jog, 
               x_jitter=.2, col="WindDirection" , height=6, aspect=1, col_wrap=3)

In [ ]:
train_jog['WindSpeed'].value_counts().head()

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Velocidade do Vento e Jardas
plt.figure(figsize=(18, 8))
sns.regplot(x='WindSpeed', y='Yards', data=train_jog, color='b', x_jitter=2)
plt.xlabel('Velocidade do Vento')
plt.ylabel('Jardas avancadas')
plt.title('Velocidade do Vento vs Jardas', fontsize=20)
plt.show()

In [ ]:
# Exibe o grafico linear com a relacao entre Velocidade do Vento, Clima e Jardas
# A maioria da umidade 0 está no clima NONE
g = sns.lmplot(x='WindSpeed', y='Yards', data=train_jog, 
               x_jitter=.2, col="GameWeather" , height=6, aspect=1)

In [ ]:
# Exibe o grafico linear com a relacao entre Velocidade do Vento, Direcao do Vento e Jardas
g = sns.lmplot(x='WindSpeed', y='Yards', data=train_jog, 
               x_jitter=.2, col="WindDirection" , height=6, aspect=1, col_wrap=3)

### Analisando as variaveis gerais da JOGADA (PlayId)
- **HomeScoreBeforePlay:** score do time da casa antes da jogada iniciar
- **VisitorScoreBeforePlay:** score do time visitante antes da jogada iniciar
- **PossessionTeam:** time com a posse
- **FieldPosition:** de que lado do campo a jogada está acontecendo
- **Quarter:** quarter do jogo (1-5, 5 = overtime)
- **PlayDirection:** direção da jogada
- **OffensePersonnel:** posição de agrupamento da equipe ofensiva
- **DefensePersonnel:** posição de agrupamento da equipe defensiva
- **OffenseFormation:** formação ofensiva
- **DefendersInTheBox:** número de defensores alinhados perto da linha de scrimmage, abrangendo a largura da linha ofensiva
- **TimeSnap:** hora (UTC) do momento que é arremessada a bola
- **TimeHandoff:** hora (UTC) do momento que o corredor pega a bola lançada
- **Down:**
- **Distance:** jardas que a equipe precisa para fazer a primeira descida
- **Dis:**
- **YardLine:** a linha de jarda da linha de scrimmage
- **X:** velocidade (em jardas/segundo)
- **A:** aceleração (em jardas/segundo^2)

In [ ]:
# Visualizando o grafico de distribuicao para cada feature do ambiente
columns_to_plot = ['HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'PossessionTeam', 'FieldPosition', 'Quarter',
                   'PlayDirection','OffensePersonnel','DefensePersonnel','OffenseFormation','DefendersInTheBox',
                   'Down','Distance','Dis','YardLine','X','A']
plot_distribution(train_jog[columns_to_plot], cols=4, width=30, height=20, hspace=0.8, wspace=0.5)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 8))

ax1.set_title('Score do Visitante antes da Jogada')
sns.regplot(x='VisitorScoreBeforePlay', y='Yards', data=train_jog, color='b', x_jitter=2, ax=ax1)

ax2.set_title('Score do Time da Casa antes da Jogada')
sns.regplot(x='HomeScoreBeforePlay', y='Yards', data=train_jog, color='g', x_jitter=2, ax=ax2)

plt.show()

In [ ]:
# Exibe o grafico de catPlot com a relacao entre PossessionTeam e Jardas
g = sns.catplot(x='PossessionTeam', y='Yards', data=train_jog, kind="box", height=8, aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
# Exibe o grafico de catPlot com a relacao entre FieldPosition e Jardas
g = sns.catplot(x='FieldPosition', y='Yards', data=train_jog, kind="box", height=8, aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
# Exibe o grafico linear com a relacao entre Distancia, Quarter e Jardas
g = sns.lmplot(x='Distance', y='Yards', data=train_jog, x_jitter=.2, col="Quarter" , height=6, aspect=1)

In [ ]:
# Exibe o grafico linear com a relacao entre Distancia, Direcao da Jogada e Jardas
g = sns.lmplot(x='Distance', y='Yards', data=train_jog, x_jitter=.2, col="PlayDirection" , height=10)

In [ ]:
# Exibe o grafico de catPlot com a relacao entre OffensePersonnel e Jardas
g = sns.catplot(x='OffensePersonnel', y='Yards', data=train_jog, kind="violin", height=8, aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
# Exibe o grafico de catPlot com a relacao entre DefensePersonnel e Jardas
g = sns.catplot(x='DefensePersonnel', y='Yards', data=train_jog, kind="violin", height=8, aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
# Exibe o grafico de catPlot com a relacao entre OffenseFormation e Jardas
g = sns.catplot(x='OffenseFormation', y='Yards', data=train_jog, kind="violin", height=8, aspect=2)
g.set_xticklabels(rotation=90)

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre DefendersInTheBox e Jardas
plt.figure(figsize=(18, 8))
sns.regplot(x='DefendersInTheBox', y='Yards', data=train_jog, color='b', x_jitter=2)
plt.xlabel('Defenders In The Box')
plt.ylabel('Jardas avancadas')
plt.title('Defenders In The Box vs Jardas', fontsize=20)
plt.show()

In [ ]:
# UTC time of the snap
train_jog['TimeSnap'].value_counts().head()

In [ ]:
# UTC time of the handoff
train_jog['TimeHandoff'].value_counts().head()

In [ ]:
# Exibe o grafico linear com a relacao entre Distancia, Down e Jardas
g = sns.lmplot(x='Distance', y='Yards', data=train_jog, x_jitter=.2, col="Down" , height=6, aspect=1)

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Distance e Jardas
plt.figure(figsize=(18, 8))
sns.regplot(x='Distance', y='Yards', data=train_jog, color='b', x_jitter=2)
plt.xlabel('Distance')
plt.ylabel('Jardas avancadas')
plt.title('Distance vs Jardas', fontsize=20)
plt.show()

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Dis e Jardas
plt.figure(figsize=(18, 8))
sns.regplot(x='Dis', y='Yards', data=train_jog, color='b', x_jitter=2)
plt.xlabel('Dis')
plt.ylabel('Jardas avancadas')
plt.title('Dis vs Jardas', fontsize=20)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 8))

ax1.set_title('Distance')
sns.regplot(x='Distance', y='Yards', data=train_jog, color='b', x_jitter=2, ax=ax1)

ax2.set_title('Dis')
sns.regplot(x='Dis', y='Yards', data=train_jog, color='g', x_jitter=2, ax=ax2)

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 8))

ax1.set_title('Aceleração')
sns.regplot(x='A', y='Yards', data=train_jog, color='b', x_jitter=2, ax=ax1)

ax2.set_title('Velocidade')
sns.regplot(x='S', y='Yards', data=train_jog, color='g', x_jitter=2, ax=ax2)

plt.show()

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Aceleração e Velocidade
plt.figure(figsize=(18, 8))
sns.regplot(x='A', y='S', data=train_jog, color='b', x_jitter=2)
plt.xlabel('Aceleração')
plt.ylabel('Velocidade')
plt.title('Aceleração vs Velocidade', fontsize=20)
plt.show()

### Analisando as variaveis de POSICIONAMENTO DO JOGADOR (player)

- **X:** posição do jogador ao longo do eixo longo do campo
- **Y:** posição do jogador ao longo do eixo curto do campo
- **Orientation:** orientação do jogador
- **Dir:** anglo do movimento do jogador

In [ ]:
# Visualizando o grafico de distribuicao para cada feature do ambiente
columns_to_plot = ['X', 'Y', 'PossessionTeam', 'Orientation', 'Dir']
plot_distribution(train_jog[columns_to_plot], cols=3, width=30, height=20, hspace=0.8, wspace=0.5)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 8))

ax1.set_title('Angulo X')
sns.regplot(x='X', y='Yards', data=train_jog, color='b', x_jitter=2, ax=ax1)

ax2.set_title('Angulo Y')
sns.regplot(x='Y', y='Yards', data=train_jog, color='g', x_jitter=2, ax=ax2)

plt.show()

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre o angulo do jogador X e Y
plt.figure(figsize=(18, 8))
sns.regplot(x='X', y='Y', data=train_jog, color='b', x_jitter=2)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Angulo do Jogador (X vs Y)', fontsize=20)
plt.show()

In [ ]:
train_jog['Orientation'].value_counts().head()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 8))

ax1.set_title('Orientation')
sns.regplot(x='Orientation', y='Yards', data=train_jog, color='b', x_jitter=2, ax=ax1)

ax2.set_title('Dir')
sns.regplot(x='Dir', y='Yards', data=train_jog, color='g', x_jitter=2, ax=ax2)

plt.show()

In [ ]:
# Exibe o grafico de scatterPlot com a relacao entre Orientation e Dir
plt.figure(figsize=(18, 8))
sns.regplot(x='Orientation', y='Dir', data=train_jog, color='b', x_jitter=2)
plt.xlabel('Orientation')
plt.ylabel('Dir')
plt.title('Orientation x Dir', fontsize=20)
plt.show()

# 7. Feature Engineering

In [ ]:
# Criando um novo dataset para criação das novas features
all_feat = train.copy()
all_feat.head()

In [ ]:
# Funcao para realizar feature engineering no dataset (treino ou teste)
def feature_engineering(df): 
    
    # Limpeza e conversao dos dados
    fill_na(df)
    convert_data(df)
    
    # Conversao de algumas features
    df['PlayerHeight']  = df['PlayerHeight'].apply(convert_to_cm)
    df['PlayerWeight']  = df['PlayerWeight'].apply(convert_to_kg)
    df['Temperature']   = df['Temperature'].apply(convert_to_celsius)
    df['StadiumType']   = df['StadiumType'].apply(agrupar_tipo_estadio)
    df['Stadium']       = df['Stadium'].apply(agrupar_estadio)
    df['Location']      = df['Location'].apply(agrupar_local)
    df['Turf']          = df['Turf'].apply(agrupar_gramado)
    df['WindDirection'] = df['WindDirection'].apply(agrupa_wind_direction)
    df['WindSpeed']     = df['WindSpeed'].apply(convert_wind_speed)
    df['GameWeather']   = df['GameWeather'].apply(agrupar_clima)
    df['GameClock']     = df['GameClock'].apply(str_to_seconds)

    # Corrigindo a feature Stadium
    df.loc[df['Stadium'] == 'MetLife Stadium', 'StadiumType'] = 'outdoor'
    df.loc[df['Stadium'] == 'StubHub Center', 'StadiumType'] = 'outdoor'    
    
    # Nova feature com a diferença entre o tempo de lançamento da bola até quando o jogador captura
    df['TimeDifer'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)            
         
    # Nova feature para indicar se é o jogador que esta realizando a jogada (corredor)
    df['IsRusher'] = df['NflId'] == df['NflIdRusher']
    
    # Cria um label encoder object
    le = preprocessing.LabelEncoder()

    # Iteracao para cada coluna do dataset de treino
    for col in df:
        if df[col].dtype == 'object':
            le.fit_transform(df[col].astype(str))
            df[col] = le.transform(df[col])      


    # Ordenacao do dataset e renovando o index
    df = df.sort_values(by=['PlayId', 'Team', 'IsRusher']).reset_index()
    
    # Removendo colunas que não serão utilizadas
    df = df.drop(['index','Yards','GameId','PlayId','NflId', 'DisplayName','NflIdRusher', 'TimeHandoff', 'TimeSnap', 'PlayerBirthDate'], axis=1)

    # Atribuindo -999 para os demais dados missing
    df.fillna(-999, inplace=True)
    
    return df

In [ ]:
# Criando um novo dataset aplicando Feature Engineering
all_feat = train.copy()
train_df = feature_engineering(all_feat)

# 8. Criação e Validação dos Modelos de ML

## Algumas considerações deste processo:
- **Cross Validation:** Estou usando 5-fold cross-validation
- **Models:** LightGBM Regression

In [ ]:
# Fazendo uma limpeza na memoria
gc.collect()

In [ ]:
# Setup cross validation folds
kf = 5
folds = KFold(n_splits=kf, shuffle=False, random_state=42)
print(str(kf) + ' Folds para treino...')

In [ ]:
# Identificando as features para o modelo
features = list(train_df.columns)
X = all_feat[features]
print(len(features),' features')
np.array(features)

In [ ]:
# Criando matrix de treino
train_data = np.zeros((509762//22,len(features))) 

for i in tqdm.tqdm(range(0,509762,22)):
    count=0
    for c in features:
        train_data[i//22][count] = train_df[c][i]
        count+=1

In [ ]:
# Split de dados e label
X_train = pd.DataFrame(data=train_data,columns=features)
y_train_ = np.array([all_feat["Yards"][i] for i in range(0,509762,22)])

In [ ]:
# Normalizacao das features 
y_tr = np.zeros(len(y_train_),dtype=np.float)
for i in range(len(y_tr)):
    y_tr[i]=(y_train_[i])

scaler = preprocessing.StandardScaler()
scaler.fit([[y] for y in y_tr])
y_tr = np.array([y[0] for y in scaler.transform([[y] for y in y_tr])])
data = [0 for i in range(199)]
for y in y_tr:
    data[int(y+99)]+=1
plt.plot([i-99 for i in range(199)],data)

## Validação
Continuous Ranked Probability Score (CRPS) is derived based on the predicted scalar value.
The CRPS is computed as follows:
$$
C=\frac{1}{199N}\sum_{m=1}^N\sum_{n=-99}^{99}(P(y\geq n)-H(n-Y_m))^2
$$
$H(x)=1$ if $x\geq 0$ else $0$

In [ ]:
def funcao_crps(labels,predictions) :
    y_pred = np.zeros((len(labels),199))
    y_ans = np.zeros((len(labels),199))
    j = np.array(range(199))
    for i,(p,t) in enumerate(zip(np.round(scaler.inverse_transform(predictions)),labels)) :
        k2 = j[j>=p-10]
        y_pred[i][k2]=(k2+10-p)*0.05
        k1 = j[j>=p+10]
        y_pred[i][k1]= 1.0
        k3 = j[j>=t]
        y_ans[i][k3]= 1.0
                           
    return 'CRPS: ', np.sum((y_pred-y_ans)**2)/(199*y_pred.shape[0]), False

In [ ]:
best_params_lgb = {
    "boosting": "gbdt",
    "verbosity": -1,
    "num_leaves":3,
    "min_data_in_leaf": 10,
    "max_depth": -1,
    "learning_rate": 0.0005,
    "bagging_freq": 4,
    "bagging_fraction": 0.1,
    "bagging_seed": 11,
    "feature_fraction" : 1,
    "random_seed": 19,
    "metric": "rmse",
    "boost_from_average" : False
}

In [ ]:
oof = np.zeros(len(X_train))
feature_importance_df = pd.DataFrame()

tr_rmse  = []
val_rmse = []
models   = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train,y_tr)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    
    X_tr, X_val = train_df.iloc[trn_idx][features], train_df.iloc[val_idx][features]
    train_y, y_val = y_tr[trn_idx], y_tr[val_idx]

    model = lgb.LGBMRegressor(**best_params_lgb, n_estimators = 100, n_jobs = -1)
    model.fit(X_tr, 
              train_y, 
              eval_set=[(X_tr, train_y), (X_val, y_val)], 
              eval_metric=funcao_crps,
              verbose=10, 
              early_stopping_rounds=99)
    
    oof[val_idx] = model.predict(X_val)
    
    val_score = mean_squared_error(y_val, oof[val_idx])
    val_rmse.append(val_score)
    
    tr_score = mean_squared_error(train_y, model.predict(X_tr))
    tr_rmse.append(tr_score)
    
    models.append(model)
    
    
    # Feature importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = X_tr.columns
    fold_importance_df["importance"] = model.feature_importances_[:len(X_tr.columns)]
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:50].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(18,18))
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('LightGBM Features Importance (media dos folds)')
plt.tight_layout()

In [ ]:
# Validacao do modelo
# Imprimir o score de treino e teste

mean_rmse_tr = np.mean(tr_rmse)
std_rmse_tr =  np.std(tr_rmse)

mean_rmse_val =  np.mean(val_rmse)
std_rmse_val =  np.std(val_rmse)

all_rmse = mean_squared_error(oof,y_tr)

print("Score de Treino")
print("Média RMSE: %.5f, std: %.5f." % (mean_rmse_tr, std_rmse_tr),'\n')

print("Score de Validação")
print("Média RMSE: %.5f, std: %.5f." % (mean_rmse_val, std_rmse_val),'\n')

print("Geral: %.5f." % (all_rmse),'\n')

print("CRPS: %.5f." % (funcao_crps(y_tr,oof)[1]))

# REALIZANDO A SUBMISSAO